<a href="https://colab.research.google.com/github/fiocruzvig/vigsaude-back/blob/main/TUB_AB_OR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### MONTAR O DRIVE ####
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### IMPORTAR DADOS DO DRIVE #####
import pandas as pd
import numpy as np

df = pd.DataFrame()

for ano in range(2019, 2023):
  df_temp = pd.read_csv(f"/content/drive/MyDrive/TUB_CN/{ano}/TUB_CN_{ano}.csv", index_col=False, low_memory=False)
  df = pd.concat([df, df_temp])
df.shape

(378297, 97)

In [ ]:

def funcao(df_aux, col ,ref):
  from scipy.stats import chi2_contingency
  total = df_aux.shape[0]
  fem_ab = df_aux[(df_aux[col] == ref) & (df_aux["ABANDONO"] == 1)].shape[0]
  result = (fem_ab/total)*100
  porc_fem_ab = round(result, 2)
  categorias = df_aux[col].unique()
  index = np.argwhere(categorias==ref)
  categorias = np.delete(categorias, index)
  df_temp = pd.DataFrame(columns=["%_ABANDONO", col, "OR(95 % CI)", "P_VALUE", "CHI**2"])
  df_temp = pd.DataFrame({"%_ABANDONO": [porc_fem_ab], col: [ref], "OR(95 % CI)":1, "P_VALUE":"", "CHI**2":""}, index=[0])

  for categoria in categorias:
    tabel_ref =  pd.crosstab(df_aux[df_aux[col] == ref]["ABANDONO"], df_aux[df_aux[col] == ref][col])
    tabel_comp = pd.crosstab(df_aux[df_aux[col] == categoria]["ABANDONO"], df_aux[df_aux[col] == categoria][col])
    tabel_cont = pd.DataFrame({ref: tabel_ref.loc[:,ref], categoria: tabel_comp.loc[:,categoria]}).T
    tabel_cont = tabel_cont[[1.0, 0.0]]
    chi2, p, _ , _= chi2_contingency(tabel_cont)
    chi2 = round(chi2, 2)
    p = round(p, 3)
    if p < 0.01:
      p = "< 0.01"
    tabel_cont = tabel_cont.iloc[::-1]
    odds = calc_odds(tabel_cont)
    result = round((tabel_cont.iloc[0,0]/total),2)*100
    df_temp = pd.concat([df_temp, pd.DataFrame({"%_ABANDONO":[result],col: [categoria], "OR(95 % CI)":[odds], "P_VALUE":[p], "CHI**2":[chi2]})])

  df_temp = df_temp.reset_index(drop=True)
  return df_temp








In [ ]:
def calc_odds(tabel_cont):
  comp_sucess = tabel_cont.iloc[0,0]
  comp_fail = tabel_cont.iloc[0,1]
  ref_sucess = tabel_cont.iloc[1,0]
  ref_fail = tabel_cont.iloc[1,1]
  OR = (comp_sucess*ref_fail)/(comp_fail*ref_sucess)
  log_or = np.log(OR)
  erro = np.sqrt(1/comp_sucess + 1/comp_fail + 1/ref_sucess + 1/ref_fail)
  z = 1.96
  CI_min = np.exp(log_or - z*erro)
  CI_max = np.exp(log_or + z*erro)
  return f"{OR:.2f} [{CI_min:.2f} - {CI_max:.2f}]"


In [ ]:
#Amostra
df = df[["CS_SEXO","NU_IDADE_N" ,"CS_RACA","CS_ESCOL_N" , "POP_LIBER", "POP_RUA","POP_SAUDE","POP_IMIG", "BENEF_GOV", "AGRAVDROGA", "AGRAVALCOO", "AGRAVTABAC" ,"AGRAVDIABE" ,"AGRAVDOENC", "AGRAVOUTRA","FORMA","HIV","TRATSUP_AT","SITUA_ENCE"]]
df.head()

,CS_SEXO,NU_IDADE_N,CS_RACA,CS_ESCOL_N,POP_LIBER,POP_RUA,POP_SAUDE,POP_IMIG,BENEF_GOV,AGRAVDROGA,AGRAVALCOO,AGRAVTABAC,AGRAVDIABE,AGRAVDOENC,AGRAVOUTRA,FORMA,HIV,TRATSUP_AT,SITUA_ENCE
0,F,4036.0,1.0,6.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,9.0,5.0
1,M,4025.0,4.0,7.0,2.0,2.0,2.0,2.0,9.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN
2,M,4043.0,4.0,5.0,1.0,2.0,2.0,2.0,9.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,1.0,1.0,3.0
3,M,4027.0,4.0,1.0,2.0,2.0,2.0,2.0,9.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,5.0
4,F,4021.0,4.0,3.0,2.0,2.0,2.0,2.0,9.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,3.0


In [ ]:
# Cópia da Amostra
df_temp = df.copy()
df_temp = df_temp.reset_index(drop=True)

In [ ]:
### ABANDONO + ÓBITO + CURA ####
df_temp = df_temp[df_temp["SITUA_ENCE"].isin([1,2,3])]
df_temp = df_temp.reset_index(drop=True)
df_temp["SITUA_ENCE"].value_counts()

,count
SITUA_ENCE,
1.0,239600
2.0,56311
3.0,15134


In [ ]:
### DROP de Nulos, Ignorados e Outros ####
df_temp = df_temp.dropna()

tam = df_temp.shape[1]
#SEXO
df_temp= df_temp[(df_temp.iloc[:,0] != "I")]

#ESCOLARIDADE
df_temp= df_temp[(df_temp.iloc[:,3] != 10)]

#BENEF_GOV
df_temp= df_temp[(df_temp.iloc[:,8] != 3) & (df_temp.iloc[:,8] != 4) & (df_temp.iloc[:,8] != 5) & (df_temp.iloc[:,8] != 7) & (df_temp.iloc[:,8] != 8)]

#HIV
df_temp= df_temp[(df_temp.iloc[:,16] != 4)]


for i in range(tam):
  df_temp = df_temp[(df_temp.iloc[:,i] != 9)]



In [ ]:
### ABANDONO + OUTROS(CURA & ÓBITO) ###
df_temp.replace({"SITUA_ENCE": {1: 0, 2: 1, 3: 0}}, inplace=True)
df_temp["SITUA_ENCE"].value_counts().sum()


79929

In [ ]:
#### SEXO #####
df_sexo = df_temp[["CS_SEXO", "SITUA_ENCE"]]
df_sexo = df_sexo.rename(columns={"CS_SEXO": "SEXO", "SITUA_ENCE": "ABANDONO"})
df_sexo.replace({"SEXO": {"M": "Masculino", "F": "Feminino", "I":"Ignorado"}}, inplace=True)
df_sexo["SEXO"].value_counts()
df_sexo["SEXO"].fillna("Nulo", inplace=True)
df_sexo = funcao(df_sexo, "SEXO", "Feminino")
df_sexo




,%_ABANDONO,SEXO,OR(95 % CI),P_VALUE,CHI**2
0,3.63,Feminino,1,,
1,11.00,Masculino,1.62 [1.55 - 1.70],< 0.01,455.63


In [ ]:
#### IDADE ####
df_idade = df_temp[["NU_IDADE_N", "SITUA_ENCE"]]
df_idade = df_idade.rename(columns={"NU_IDADE_N": "FAIXA_ETARIA", "SITUA_ENCE": "ABANDONO"})
df_idade["FAIXA_ETARIA"] = pd.cut(df_idade["FAIXA_ETARIA"],
                                  bins=[-np.inf, 4019, 4039, 4059, np.inf],
                                  labels=[1, 2, 3, 4])

if pd.api.types.is_categorical_dtype(df_idade["FAIXA_ETARIA"]):
    current_categories = df_idade["FAIXA_ETARIA"].cat.categories
    if 0 not in current_categories:
        df_idade["FAIXA_ETARIA"] = df_idade["FAIXA_ETARIA"].cat.add_categories([0])

df_idade["FAIXA_ETARIA"].fillna(0, inplace=True)
df_idade.replace({"FAIXA_ETARIA": {0: "Nulos", 1: "<=19 anos", 2: "20-39 anos", 3: "40-59 anos", 4: "60+ anos"}}, inplace=True)
df_idade = funcao(df_idade, "FAIXA_ETARIA", "<=19 anos")
df_idade


<ipython-input-627-bacfa0547ad5>:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_idade["FAIXA_ETARIA"]):


,%_ABANDONO,FAIXA_ETARIA,OR(95 % CI),P_VALUE,CHI**2
0,0.97,<=19 anos,1,,
1,1.00,60+ anos,0.44 [0.40 - 0.49],< 0.01,229.58
2,9.00,20-39 anos,1.59 [1.47 - 1.72],< 0.01,131.32
3,4.00,40-59 anos,1.07 [0.98 - 1.16],0.119,2.43


In [ ]:
### RAÇA ###
df_raca = df_temp.copy()
df_raca["CS_RACA"] = np.where(df_raca["CS_RACA"].between(2,5), 2, df_raca["CS_RACA"])
df_raca["CS_RACA"].fillna(0, inplace=True)
df_raca = df_raca[["CS_RACA", "SITUA_ENCE"]]
df_raca = df_raca.rename(columns={"CS_RACA": "RAÇA", "SITUA_ENCE": "ABANDONO"})
df_raca.replace({"RAÇA": {0: "Nulos", 1: "Branca", 2: "Não_Branca", 9:"Ignorado"}}, inplace=True)
df_raca = funcao(df_raca, "RAÇA", "Branca")
df_raca


,%_ABANDONO,RAÇA,OR(95 % CI),P_VALUE,CHI**2
0,3.66,Branca,1,,
1,11.00,Não_Branca,1.32 [1.26 - 1.38],< 0.01,146.47


In [ ]:
### ESCOLARIDADE ####
df_esc = df_temp.copy()
df_esc["CS_ESCOL_N"] = pd.cut(df_esc["CS_ESCOL_N"], bins=[-np.inf, 0, 4,8,9,np.inf],
                                  labels=[1,2,3,4,5])
df_esc = df_esc[["CS_ESCOL_N", "SITUA_ENCE"]]
df_esc = df_esc.rename(columns={"CS_ESCOL_N": "ESCOLARIDADE", "SITUA_ENCE": "ABANDONO"})


if pd.api.types.is_categorical_dtype(df_esc["ESCOLARIDADE"]):
    current_categories =df_esc["ESCOLARIDADE"].cat.categories
    if 0 not in current_categories:
       df_esc["ESCOLARIDADE"] = df_esc["ESCOLARIDADE"].cat.add_categories([0])

df_esc["ESCOLARIDADE"].fillna(0, inplace=True)
df_esc.replace({"ESCOLARIDADE": {0: "Nulos", 1: "Sem_Escolaridade", 2: "<=8_anos", 3: ">8_anos", 4:"Ignorado", 5:"Não_aplicado"}}, inplace=True)
df_esc = funcao(df_esc, "ESCOLARIDADE", ">8_anos")
df_esc




<ipython-input-629-7fdca2774d69>:9: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_esc["ESCOLARIDADE"]):


,%_ABANDONO,ESCOLARIDADE,OR(95 % CI),P_VALUE,CHI**2
0,4.65,>8_anos,1,,
1,1.00,Sem_Escolaridade,1.06 [0.96 - 1.18],0.243,1.36
2,10.00,<=8_anos,1.65 [1.58 - 1.72],< 0.01,553.41


In [ ]:
### PRISIONEIROS ####
df_pri = df_temp.copy()
df_pri.replace({"POP_LIBER": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_pri['POP_LIBER'].fillna("Nulos", inplace=True)
df_pri = df_pri.rename(columns={"POP_LIBER": "PRISIONEIROS", "SITUA_ENCE": "ABANDONO"})
df_pri = funcao(df_pri, "PRISIONEIROS", "Não")
df_pri


,%_ABANDONO,PRISIONEIROS,OR(95 % CI),P_VALUE,CHI**2
0,13.96,Não,1,,
1,1.00,Sim,0.71 [0.66 - 0.76],< 0.01,81.21


In [ ]:
### POP_RUA ####
df_rua = df_temp.copy()
df_rua.replace({"POP_RUA": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_rua['POP_RUA'].fillna("Nulos", inplace=True)
df_rua = df_rua.rename(columns={"POP_RUA": "POP_RUA", "SITUA_ENCE": "ABANDONO"})
df_rua = funcao(df_rua, "POP_RUA", "Não")
df_rua

,%_ABANDONO,POP_RUA,OR(95 % CI),P_VALUE,CHI**2
0,13.63,Não,1,,
1,1.00,Sim,6.47 [5.93 - 7.05],< 0.01,2281.49


In [ ]:
### PROFISSIONAIS DA SAÚDE ####
df_saude = df_temp.copy()
df_saude.replace({"POP_SAUDE": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_saude["POP_SAUDE"].fillna("Nulos", inplace=True)
df_saude = df_saude.rename(columns={"POP_SAUDE": "PROFISSIONAIS_SAUDE", "SITUA_ENCE": "ABANDONO"})
df_saude = funcao(df_saude, "PROFISSIONAIS_SAUDE", "Não")
df_saude

,%_ABANDONO,PROFISSIONAIS_SAUDE,OR(95 % CI),P_VALUE,CHI**2
0,14.91,Não,1,,
1,0.00,Sim,0.33 [0.26 - 0.41],< 0.01,105.2


In [ ]:
### IMIGRANTE ###
df_imig = df_temp.copy()
df_imig.replace({"POP_IMIG": {1: "Sim", 2: "Não", 3:"Outros",9:"Ignorado"}}, inplace=True)
df_imig["POP_IMIG"].fillna("Nulos", inplace=True)
df_imig = df_imig.rename(columns={"POP_IMIG": "IMIGRANTE", "SITUA_ENCE": "ABANDONO"})
df_imig = funcao(df_imig, "IMIGRANTE", "Não")
df_imig


,%_ABANDONO,IMIGRANTE,OR(95 % CI),P_VALUE,CHI**2
0,14.92,Não,1,,
1,0.00,Sim,1.12 [0.87 - 1.45],0.409,0.68


In [ ]:
#### BENEFICIÁRIO #####
df_benef = df_temp.copy()
df_benef.replace({"BENEF_GOV": {1: "Sim", 2: "Não", 5: "Outros", 7: "Outros",3:"Outros", 4:"Outros", 8:"Outros",9:"Ignorado"}}, inplace=True)
df_benef["BENEF_GOV"].fillna("Nulos", inplace=True)
df_benef = df_benef.rename(columns={"BENEF_GOV": "BENEFICIARIO", "SITUA_ENCE": "ABANDONO"})
df_benef = funcao(df_benef, "BENEFICIARIO", "Não")
df_benef


,%_ABANDONO,BENEFICIARIO,OR(95 % CI),P_VALUE,CHI**2
0,13.44,Não,1,,
1,2.00,Sim,0.92 [0.87 - 0.98],0.015,5.89


In [ ]:
### USUÁRIO DE DROGAS ###
df_droga = df_temp.copy()
df_droga.replace({"AGRAVDROGA": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_droga["AGRAVDROGA"].fillna("Nulos", inplace=True)
df_droga  = df_droga.rename(columns={"AGRAVDROGA": "DROGAS", "SITUA_ENCE": "ABANDONO"})
df_droga = funcao(df_droga, "DROGAS", "Não")
df_droga

,%_ABANDONO,DROGAS,OR(95 % CI),P_VALUE,CHI**2
0,9.77,Não,1,,
1,5.00,Sim,3.73 [3.57 - 3.89],< 0.01,3795.52


In [ ]:
### TABACO ###
df_tab = df_temp.copy()
df_tab.replace({"AGRAVTABAC": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_tab["AGRAVTABAC"].fillna("Nulos", inplace=True)
df_tab  = df_tab.rename(columns={"AGRAVTABAC": "TABACO", "SITUA_ENCE": "ABANDONO"})
df_tab = funcao(df_tab, "TABACO", "Não")
df_tab

,%_ABANDONO,TABACO,OR(95 % CI),P_VALUE,CHI**2
0,8.85,Não,1,,
1,6.00,Sim,2.27 [2.18 - 2.36],< 0.01,1632.74


In [ ]:
### ALCOOL ###
df_alco = df_temp.copy()
df_alco.replace({"AGRAVALCOO": {1: "Sim", 2: "Não",9:"Ignorado"}}, inplace=True)
df_alco["AGRAVALCOO"].fillna("Nulos", inplace=True)
df_alco  = df_alco.rename(columns={"AGRAVALCOO": "ALCOOL", "SITUA_ENCE": "ABANDONO"})
df_alco = funcao(df_alco, "ALCOOL", "Não")
df_alco

,%_ABANDONO,ALCOOL,OR(95 % CI),P_VALUE,CHI**2
0,10.49,Não,1,,
1,5.00,Sim,2.28 [2.18 - 2.38],< 0.01,1394.65


In [ ]:
### DIABETES ###
df_dia = df_temp.copy()
df_dia.replace({"AGRAVDIABE": {1: "Sim",2: "Não",9:"Ignorado"}}, inplace=True)
df_dia["AGRAVDIABE"].fillna("Nulos", inplace=True)
df_dia  = df_dia.rename(columns={"AGRAVDIABE": "DIABETES", "SITUA_ENCE": "ABANDONO"})
df_dia = funcao(df_dia, "DIABETES", "Não")
df_dia

,%_ABANDONO,DIABETES,OR(95 % CI),P_VALUE,CHI**2
0,14.25,Não,1,,
1,1.00,Sim,0.48 [0.44 - 0.52],< 0.01,293.2


In [ ]:
### PROBLEMAS MENTAIS ###
df_ment = df_temp.copy()
df_ment.replace({"AGRAVDOENC": {1: "Sim",2: "Não",9:"Ignorado"}}, inplace=True)
df_ment["AGRAVDOENC"].fillna("Nulos", inplace=True)
df_ment  = df_ment.rename(columns={"AGRAVDOENC": "PROBLEMAS_MENTAIS", "SITUA_ENCE": "ABANDONO"})
df_ment = funcao(df_ment, "PROBLEMAS_MENTAIS", "Não")
df_ment


,%_ABANDONO,PROBLEMAS_MENTAIS,OR(95 % CI),P_VALUE,CHI**2
0,14.49,Não,1,,
1,1.00,Sim,1.45 [1.30 - 1.62],< 0.01,44.21


In [ ]:
### OUTRAS DOENÇAS ###
df_out = df_temp.copy()
df_out.replace({"AGRAVOUTRA": {1: "Sim",2: "Não",9:"Ignorado"}}, inplace=True)
df_out["AGRAVOUTRA"].fillna("Nulos", inplace=True)
df_out  = df_out.rename(columns={"AGRAVOUTRA": "OUTRAS_DOENCAS", "SITUA_ENCE": "ABANDONO"})
df_out = funcao(df_out, "OUTRAS_DOENCAS", "Não")
df_out

,%_ABANDONO,OUTRAS_DOENCAS,OR(95 % CI),P_VALUE,CHI**2
0,13.49,Não,1,,
1,2.00,Sim,0.84 [0.79 - 0.90],< 0.01,26.89


In [ ]:
### FORMAS ###
df_forma = df_temp.copy()
df_forma.replace({"FORMA": {1: "Pulmonar", 2: "Extrapulmonar", 3:"Pulmonar_Extrapulmonar"}}, inplace=True)
df_forma["FORMA"].fillna("Nulos", inplace=True)
df_forma  = df_forma.rename(columns={"SITUA_ENCE": "ABANDONO"})
df_forma = funcao(df_forma, "FORMA", "Pulmonar")
df_forma

,%_ABANDONO,FORMA,OR(95 % CI),P_VALUE,CHI**2
0,13.6,Pulmonar,1,,
1,1.0,Extrapulmonar,0.59 [0.55 - 0.63],< 0.01,200.21
2,0.0,Pulmonar_Extrapulmonar,0.97 [0.86 - 1.11],0.706,0.14


In [ ]:
### HIV ####
df_hiv = df_temp.copy()
df_hiv.replace({"HIV": {1: "Sim", 2: "Não", 3:"Em_Andamento", 4:"Não_realizado"}}, inplace=True)
df_hiv["HIV"].fillna("Nulos", inplace= True)
df_hiv  = df_hiv.rename(columns={"SITUA_ENCE": "ABANDONO"})
df_hiv = funcao(df_hiv, "HIV", "Não")
df_hiv

,%_ABANDONO,HIV,OR(95 % CI),P_VALUE,CHI**2
0,12.79,Não,1,,
1,2.00,Sim,2.38 [2.25 - 2.53],< 0.01,871.7
2,0.00,Em_Andamento,1.01 [0.72 - 1.42],1.0,0.0


In [ ]:
### SUPERVISIONADO ####
df_sup = df_temp.copy()
df_sup.replace({"TRATSUP_AT": {1: "Sim",2: "Não",9:"Ignorado"}}, inplace=True)
df_sup["TRATSUP_AT"].fillna("Nulos", inplace=True)
df_sup  = df_sup.rename(columns={"TRATSUP_AT": "SUPERVISIONADO", "SITUA_ENCE": "ABANDONO"})
df_sup = funcao(df_sup, "SUPERVISIONADO", "Não")
df_sup

,%_ABANDONO,SUPERVISIONADO,OR(95 % CI),P_VALUE,CHI**2
0,9.85,Não,1,,
1,5.00,Sim,0.51 [0.49 - 0.53],< 0.01,1103.26


In [ ]:
dfs = []
dfs.append(df_sexo)
dfs.append(df_idade)
dfs.append(df_raca)
dfs.append(df_esc)
dfs.append(df_pri)
dfs.append(df_rua)
dfs.append(df_saude)
dfs.append(df_imig)
dfs.append(df_benef)
dfs.append(df_droga)
dfs.append(df_tab)
dfs.append(df_alco)
dfs.append(df_dia)
dfs.append(df_ment)
dfs.append(df_out)
dfs.append(df_forma)
dfs.append(df_hiv)
dfs.append(df_sup)
len(dfs)

18

In [ ]:
### GERAR PLANILHAS ###
for d in dfs:
  d.to_excel(f"/content/{d.columns[1]}_AB.xlsx", index=False)

In [ ]:
import zipfile
import os
from google.colab import files

directory = '/content/'
zip_filename = f'/content/arquivos_xlsx.zip'
arquivos = []

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(directory):
        if root != directory:
          break
        for file in files:
            if file.endswith('.xlsx'):
                arquivos.append(os.path.join(root, file))
                zipf.write(os.path.join(root, file), arcname=file)

print(f"Arquivo ZIP criado: {zip_filename}")

Arquivo ZIP criado: /content/arquivos_xlsx.zip


In [ ]:
for arquivo in arquivos:
  if os.path.exists(arquivo):
    os.remove(arquivo)
    print(f"Diretório removido: {arquivo}")
  else:
     print(f"Diretório não removido: {arquivo}")

Diretório removido: /content/DROGAS_AB.xlsx
Diretório removido: /content/OUTRAS_DOENCAS_AB.xlsx
Diretório removido: /content/DIABETES_AB.xlsx
Diretório removido: /content/FORMA_AB.xlsx
Diretório removido: /content/TABACO_AB.xlsx
Diretório removido: /content/SEXO_AB.xlsx
Diretório removido: /content/PRISIONEIROS_AB.xlsx
Diretório removido: /content/IMIGRANTE_AB.xlsx
Diretório removido: /content/HIV_AB.xlsx
Diretório removido: /content/PROFISSIONAIS_SAUDE_AB.xlsx
Diretório removido: /content/POP_RUA_AB.xlsx
Diretório removido: /content/FAIXA_ETARIA_AB.xlsx
Diretório removido: /content/BENEFICIARIO_AB.xlsx
Diretório removido: /content/PROBLEMAS_MENTAIS_AB.xlsx
Diretório removido: /content/SUPERVISIONADO_AB.xlsx
Diretório removido: /content/RAÇA_AB.xlsx
Diretório removido: /content/ESCOLARIDADE_AB.xlsx
Diretório removido: /content/ALCOOL_AB.xlsx
